In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from scipy.interpolate import make_interp_spline
from IPython.display import HTML

In [2]:
header_row = 1  # Use the second row as the column names
skip_rows = [2, 3]  # Skip the first row, and the third and fourth rows

# Read the CSV file with the specified header row and skipping the specified rows
data1 = pd.read_csv("G:\Shared drives\FMA-G\C-CONSERVACIÓN\C1-BOSQUE PEHUEN\C1.5_DATOS\Estacion Meteorológica\Linea de tiempo completa\CR800Series_Table1_13092019_21012021.dat", header=header_row, skiprows=skip_rows, low_memory=False)
data2 = pd.read_csv("G:\Shared drives\FMA-G\C-CONSERVACIÓN\C1-BOSQUE PEHUEN\C1.5_DATOS\Estacion Meteorológica\Linea de tiempo completa\CR800Series_Table 21092018_30012020.dat", header=header_row, skiprows=skip_rows, low_memory=False)
data3 = pd.read_csv("G:\Shared drives\FMA-G\C-CONSERVACIÓN\C1-BOSQUE PEHUEN\C1.5_DATOS\Estacion Meteorológica\Linea de tiempo completa\CR800Series_Table 14012023_24052024.dat", header=header_row, skiprows=skip_rows, low_memory=False)
data4 = pd.read_csv("G:\Shared drives\FMA-G\C-CONSERVACIÓN\C1-BOSQUE PEHUEN\C1.5_DATOS\Estacion Meteorológica\Linea de tiempo completa\CR800Series_Table 08112022_18032024.dat", header=header_row, skiprows=skip_rows, low_memory=False)
data5 = pd.read_csv("G:\Shared drives\FMA-G\C-CONSERVACIÓN\C1-BOSQUE PEHUEN\C1.5_DATOS\Estacion Meteorológica\Linea de tiempo completa\CR800Series_Table 03082021_12122022.dat", header=header_row, skiprows=skip_rows, low_memory=False)
data6 = pd.read_csv("G:\Shared drives\FMA-G\C-CONSERVACIÓN\C1-BOSQUE PEHUEN\C1.5_DATOS\Estacion Meteorológica\Linea de tiempo completa\CR800Series_Table 01122020_11042022.dat", header=header_row, skiprows=skip_rows, low_memory=False)

In [3]:
dataframes = [data1, data2, data3, data4, data5, data6]

# Concatenate all dataframes
merged_df = pd.concat(dataframes)

# Drop duplicates based on 'RECORD' and 'TIMESTAMP' columns
merged_df = merged_df.drop_duplicates(subset=['RECORD', 'TIMESTAMP'])

In [4]:
# Convert the column to TIMESTAMP format
merged_df['TIMESTAMP'] = pd.to_datetime(merged_df['TIMESTAMP'])

# Extract year, month, day, and time
merged_df['Year'] = merged_df['TIMESTAMP'].dt.year
merged_df['Month'] = merged_df['TIMESTAMP'].dt.month
merged_df['Day'] = merged_df['TIMESTAMP'].dt.day
merged_df['Time'] = merged_df['TIMESTAMP'].dt.time


In [9]:
print (merged_df.columns)

Index(['TIMESTAMP', 'RECORD', 'AirTC_Max', 'AirTC_Avg', 'AirTC_Min', 'RH_Max',
       'RH_Avg', 'RH_Min', 'WS_ms_Max', 'WS_ms_Avg', 'WS_ms_Min',
       'WindDir_Max', 'WindDir_Avg', 'WindDir_Min', 'WindDir_Std',
       'BP_mbar_Avg', 'T107_10cm_Max', 'T107_10cm_Avg', 'T107_10cm_Min',
       'T107_10cm_Std', 'T107_50cm_Max', 'T107_50cm_Avg', 'T107_50cm_Min',
       'T107_50cm_Std', 'Rain_mm_Tot', 'PTemp_C_Avg', 'PtoRocio_Avg',
       'BattV_Min', 'DT_Max', 'DT_Avg', 'DT_Min', 'Q_Max', 'Q_Min', 'TCDT_Max',
       'TCDT_Min', 'incomingLW_Avg', 'incomingSW_Avg', 'outgoingLW_Avg',
       'outgoingSW_Avg', 'albedo_Avg', 'Year', 'Month', 'Day', 'Time', 'Date',
       'Temp_Norm', 'Wind_Norm', 'Hum_Norm', 'Combined', 'Is_Rainy',
       'Days_Without_Rain'],
      dtype='object')


In [5]:
# Create a datetime column for easier manipulation
merged_df['Date'] = pd.to_datetime(merged_df[['Year', 'Month', 'Day']])

# Normalize temperature and wind speed (range 0-1)
merged_df['Temp_Norm'] = (merged_df['AirTC_Avg'] - merged_df['AirTC_Avg'].min()) / (merged_df['AirTC_Avg'].max() - merged_df['AirTC_Avg'].min())
merged_df['Wind_Norm'] = (merged_df['WS_ms_Avg'] - merged_df['WS_ms_Avg'].min()) / (merged_df['WS_ms_Avg'].max() - merged_df['WS_ms_Avg'].min())

# Invert and normalize humidity (higher humidity closer to center)
merged_df['Hum_Norm'] = 1 - (merged_df['RH_Avg'] - merged_df['RH_Avg'].min()) / (merged_df['RH_Avg'].max() - merged_df['RH_Avg'].min())

# Create a new column that combines normalized values
merged_df['Combined'] = merged_df[['Temp_Norm', 'Hum_Norm', 'Wind_Norm']].mean(axis=1)

In [11]:
daily_rainfall = merged_df.groupby(merged_df['Date'].dt.date)['Rain_mm_Tot'].sum().reset_index()

# Rename columns for clarity
daily_rainfall.columns = ['Date', 'Total_Rain_mm']

# Print the daily rainfall to debug and ensure it has been created correctly
print("Daily Rainfall DataFrame:")
print(daily_rainfall.head())

# Determine if it's a rainy day (more than 2 mm of rain)
daily_rainfall['Is_Rainy'] = daily_rainfall['Total_Rain_mm'].apply(lambda x: 1 if x > 2 else 0)

# Print the Is_Rainy column to debug
print("Daily Rainfall with Is_Rainy:")
print(daily_rainfall.head())

# Convert 'Date' column in daily_rainfall back to datetime to match merged_df
daily_rainfall['Date'] = pd.to_datetime(daily_rainfall['Date'])

# Merge back into the original merged_df on the 'Date' column
merged_df = pd.merge(merged_df, daily_rainfall[['Date', 'Is_Rainy']], on='Date', how='left')

# Create a column for the cumulative count of days without rain
merged_df['Days_Without_Rain'] = (merged_df['Is_Rainy'] == 0).cumsum()

# Print the updated DataFrame to verify the changes
print("Merged DataFrame:")
print(merged_df[['Date', 'Rain_mm_Tot', 'Is_Rainy', 'Days_Without_Rain']].head())

Daily Rainfall DataFrame:
         Date  Total_Rain_mm
0  2018-09-21           13.5
1  2018-09-22           28.6
2  2018-09-23           12.3
3  2018-09-24            6.9
4  2018-09-25            0.0
Daily Rainfall with Is_Rainy:
         Date  Total_Rain_mm  Is_Rainy
0  2018-09-21           13.5         1
1  2018-09-22           28.6         1
2  2018-09-23           12.3         1
3  2018-09-24            6.9         1
4  2018-09-25            0.0         0
Merged DataFrame:
        Date  Rain_mm_Tot  Is_Rainy  Days_Without_Rain
0 2019-09-13          0.0         0                  1
1 2019-09-13          0.0         0                  2
2 2019-09-14          0.0         1                  2
3 2019-09-14          0.0         1                  2
4 2019-09-14          0.0         1                  2


In [13]:
def calculate_scores(row): 
    # Temperature score
    temperature_score = calculate_temperature_score(row['AirTC_Avg'])
    
    # Humidity score
    humidity_score = calculate_humidity_score(row['RH_Avg'])
    
    # Wind speed score
    wind_speed_score = calculate_wind_speed_score(row['WS_ms_Avg'])
    
    # Days without rain score
    days_without_rain_score = calculate_days_without_rain_score(row['Days_Without_Rain'])
    
    # Total fire risk index
    total_score = temperature_score + humidity_score + wind_speed_score + days_without_rain_score
    
    return pd.Series([temperature_score, humidity_score, wind_speed_score, days_without_rain_score, total_score])

# Functions for calculating individual scores
def calculate_temperature_score(temperature):
    if temperature < 0:
        return 2.7
    elif 0 <= temperature < 6:
        return 5.4
    elif 6 <= temperature < 11:
        return 8.1
    elif 11 <= temperature < 16:
        return 10.8
    elif 16 <= temperature < 21:
        return 13.5
    elif 21 <= temperature < 26:
        return 16.2
    elif 26 <= temperature < 31:
        return 18.9
    elif 31 <= temperature < 35:
        return 21.6
    else:
        return 25

def calculate_humidity_score(humidity):
    if 0 <= humidity < 11:
        return 25
    elif 11 <= humidity < 21:
        return 22.5
    elif 21 <= humidity < 31:
        return 20
    elif 31 <= humidity < 41:
        return 17.5
    elif 41 <= humidity < 51:
        return 15
    elif 51 <= humidity < 61:
        return 12.5
    elif 61 <= humidity < 71:
        return 10
    elif 71 <= humidity < 81:
        return 7.5
    elif 81 <= humidity < 91:
        return 5
    else:
        return 2.5

def calculate_wind_speed_score(wind_speed):
    if wind_speed < 1:
        return 3.125
    elif 1 <= wind_speed < 6:
        return 6.25
    elif 6 <= wind_speed < 11:
        return 9.375
    elif 11 <= wind_speed < 16:
        return 12.5
    elif 16 <= wind_speed < 21:
        return 15.625
    elif 21 <= wind_speed < 26:
        return 18.75
    elif 26 <= wind_speed < 31:
        return 21.875
    else:
        return 25

def calculate_days_without_rain_score(days_without_rain):
    if days_without_rain < 1:
        return 2.5
    elif 1 <= days_without_rain < 6:
        return 5
    elif 6 <= days_without_rain < 11:
        return 7.5
    elif 11 <= days_without_rain < 16:
        return 10
    elif 16 <= days_without_rain < 21:
        return 12.5
    elif 21 <= days_without_rain < 26:
        return 15
    elif 26 <= days_without_rain < 31:
        return 17.5
    elif 31 <= days_without_rain < 36:
        return 20
    elif 36 <= days_without_rain < 41:
        return 22.5
    else:
        return 25

# Apply scoring function to the DataFrame
merged_df[['Temperature_Score', 'Humidity_Score', 'Wind_Speed_Score', 'Days_Without_Rain_Score', 'Total_Score']] = merged_df.apply(calculate_scores, axis=1)


In [6]:
# Function to extract the unique dates
unique_dates = merged_df['Date'].unique()

In [ ]:
# Set up the figure and polar axis
fig, ax = plt.subplots(figsize=(6, 6), subplot_kw={'projection': 'polar'})

# Angles for temperature, humidity, and wind speed
angles = np.linspace(0, 2 * np.pi, 3, endpoint=False).tolist()
angles += angles[:1]  # Close the loop

In [ ]:
print(f"Angles Length: {len(angles)}")  # Should be 4


In [9]:
def smooth_data(values, smooth_factor=300):
    # Ensure the angles array has the correct length
    num_points = len(values)  # This should match the number of angles (4)
    
    # Create a strictly increasing array for interpolation, excluding the last angle to avoid wraparound
    temp_angles = np.linspace(0, 2 * np.pi, num_points - 1, endpoint=False)
    
    # Check lengths again
    print(f"Temp Angles Length for Interpolation: {len(temp_angles)}")  # Should be 3
    print(f"Values Length: {len(values[:-1])}")  # Should be 3 (exclude the last value for interpolation)
    
    # Ensure there are at least 3 unique points for quadratic (k=2) or 4 for cubic (k=3) interpolation
    unique_values = len(np.unique(values[:-1]))
    if unique_values < 3:
        print("Error: Not enough unique points for interpolation.")
        return temp_angles, np.zeros_like(temp_angles)  # Return a dummy result
    
    # Set degree of spline (k=2 for quadratic if < 4 unique points)
    k = 2 if unique_values == 3 else 3
    
    # Extend angles for smoothing
    angles_ext = np.linspace(0, 2 * np.pi, (num_points - 1) * smooth_factor)  # Only 3 values

    # Perform interpolation with strictly increasing x-values
    spline = make_interp_spline(temp_angles, values[:-1], k=k)  # Use dynamic k
    smooth_values = spline(angles_ext)

    # Return the smooth data with no duplication
    return angles_ext, smooth_values

In [10]:
# Initialize the line
line, = ax.plot([], [], linewidth=2)


In [11]:
def update(date):
    # Filter data for the current date
    daily_data = merged_df[merged_df['Date'] == date]

    # Values to plot
    values = [
        daily_data['Temp_Norm'].values[0],
        daily_data['Hum_Norm'].values[0],
        daily_data['Wind_Norm'].values[0]
    ]
    values += values[:1]  # Close the loop, adding the first value to the end

    # Check lengths before plotting
    print(f"Update Values Length: {len(values)}")  # Should be 4

    # Smooth values
    smooth_angles, smooth_values = smooth_data(values)

    # Close the loop for plotting, including the last value again to match radar plot
    smooth_angles = np.append(smooth_angles, smooth_angles[0])
    smooth_values = np.append(smooth_values, smooth_values[0])

    # Print smooth values for debugging
    print(f"Smooth Angles: {smooth_angles}")
    print(f"Smooth Values: {smooth_values}")

    # Change line color based on condition
    if daily_data['Temp_Norm'].values[0] > 0.7 and daily_data['Wind_Norm'].values[0] > 0.5 and daily_data['Hum_Norm'].values[0] < 0.3:
        line_color = 'red'
    else:
        line_color = 'blue'

    # Update the line properties with the correct data
    line.set_data(smooth_angles, smooth_values)
    line.set_color(line_color)

    # Set the background color
    ax.set_facecolor('lightgrey' if line_color == 'blue' else 'pink')

    # Set the title to the current date
    ax.set_title(f"Date: {date.strftime('%Y-%m-%d')}", fontsize=16)

    # Ensure line is updated properly
    return line,

In [ ]:
# Set the limits and labels for the plot
ax.set_ylim(0, 1)
ax.set_xticks(angles[:-1])  # Remove the last duplicate angle for labels
ax.set_xticklabels(['Temperature', 'Humidity', 'Wind Speed'])


In [ ]:
# Create animation
ani = FuncAnimation(fig, update, frames=unique_dates, interval=200, blit=True)  # Use blit=True for faster updates

# Display animation in notebook
HTML(ani.to_jshtml())